In [5]:
import requests
import pandas as pd
from typing import Dict, List
import time
from functools import lru_cache

# 1. Enhanced Configuration
class Config:
    BASE_URL = "https://dlmm-api.meteora.ag"
    MIN_TVL = 7000  # In USD
    VOLATILITY_WINDOW = "24h"  # For CoinGecko data
    IL_RISK_THRESHOLD = 0.05  # 5% max acceptable IL risk

# 2. Pagination Handling (Get all pools)
def fetch_all_pages(initial_params: Dict) -> List[Dict]:
    all_pairs = []
    page = 0
    
    while True:
        params = initial_params.copy()
        params.update({"page": page})
        
        try:
            resp = requests.get(
                f"{Config.BASE_URL}/pair/all_with_pagination",
                params=params,
                timeout=15
            )
            resp.raise_for_status()
            data = resp.json()
            
            if not data['pairs']:
                break
                
            all_pairs.extend(data['pairs'])
            page += 1
            
            # Rate limiting protection
            time.sleep(0.5)
            
        except requests.exceptions.HTTPError as e:
            print(f"Error fetching page {page}: {str(e)}")
            break
            
    return all_pairs

# 3. Volatility Integration (CoinGecko/Pyth)
@lru_cache(maxsize=100)
def get_token_volatility(token_id: str) -> float:
    """Fetch annualized volatility for a token"""
    try:
        url = f"https://api.coingecko.com/api/v3/coins/{token_id}/market_chart?vs_currency=usd&days=1"
        prices = [p[1] for p in requests.get(url).json()['prices']]
        returns = pd.Series(prices).pct_change().dropna()
        return returns.std() * np.sqrt(365)  # Annualized
    except Exception as e:
        print(f"Volatility fetch failed for {token_id}: {str(e)}")
        return 0.0

# 4. IL Risk Calculation (DLMM-specific)
def calculate_dlmm_il_risk(bin_step: int, current_price: float) -> float:
    """
    DLMM-specific IL estimation based on:
    - Bin step (price granularity)
    - Current price position within bins
    """
    # Simplified example - replace with actual DLMM IL formula
    price_range = bin_step * 0.01  # Assuming 1% per bin
    return min(abs(price_range - current_price % price_range)/price_range)

# 5. Enhanced Data Processing
def process_pairs_data(raw_pairs: List[Dict]) -> pd.DataFrame:
    df = pd.DataFrame(raw_pairs)
    
    # Column processing
    metric_cols = ['fees', 'fee_tvl_ratio', 'volume']
    for col in metric_cols:
        df[f"{col}_30m"] = df[col].apply(lambda x: x.get('min_30', 0) if isinstance(x, dict) else 0)
        df[f"{col}_1h"] = df[col].apply(lambda x: x.get('hour_1', 0) if isinstance(x, dict) else 0)
    
    # Proper APR calculation (annualized)
    df['liquidity'] = pd.to_numeric(df['liquidity'], errors='coerce').fillna(0)
    df['apr_30m'] = (df['fees_30m'] * 48 * 365) / df['liquidity'].replace(0, 1)
    
    # Add volatility data (example for SOL)
    df['volatility'] = df['mint_x'].apply(
        lambda x: get_token_volatility('solana') if 'SOL' in x else 0.0
    )
    
    # IL Risk estimation
    # Ensure 'price' column exists
    if 'price' not in df.columns:
        df['price'] = 0  # or some default value or calculation

    df['il_risk'] = df.apply(
        lambda row: calculate_dlmm_il_risk(row['bin_step'], row['price']),
        axis=1
    )
    
    return df

# 6. Automated Filtering
def filter_pools(df: pd.DataFrame) -> pd.DataFrame:
    return df[
        (df['liquidity'] >= Config.MIN_TVL) &
        (df['apr_30m'] > 0.15) &  # 15% minimum APR
        (df['volatility'] < 0.4) &
        (df['il_risk'] < Config.IL_RISK_THRESHOLD)
    ].sort_values('apr_30m', ascending=False)

# Main Execution Flow
if __name__ == "__main__":
    # Fetch all pools
    params = {
        "hide_low_tvl": Config.MIN_TVL,
        "sort_key": "feetvlratio30m",
        "order_by": "desc"
    }
    
    all_pairs = fetch_all_pages(params)
    processed_df = process_pairs_data(all_pairs)
    filtered_pools = filter_pools(processed_df)
    
    print(f"Top {len(filtered_pools)} Pools:")
    print(filtered_pools[['name', 'apr_30m', 'volatility', 'il_risk']].head(20))

TypeError: 'float' object is not iterable